In [53]:
import pandas as pd
import spacy
import random

In [137]:
data = pd.read_excel("ner_dataset.csv/ner_dataset.xlsx")

In [138]:
data

,SENTENCE,WORD,ENTITY
0,THANK YOU FOR CALLING WINDSTREAM PRAVEEN HOW ...,"WINDSTREAM,PRAVEEN,TYLER","ORG,PERSON,PERSON"
1,THANK YOU FOR CALLING WINDSTREAM MY NAME IS RA...,"WINDSTREAM,RAVI","ORG,PERSON"
2,THANK YOU FOR CALLING WINDSTREAM MY NAME IS VI...,"WINDSTREAM,VISHAL","ORG,PERSON"
3,THANK YOU FOR CALLING WINDSTREAM MY NAME IS HA...,"WINDSTREAM,HANDA","ORG,PERSON"
4,THANK YOU FOR CALLING WINDSTREAM MY NAME IS RO...,"WINDSTREAM,ROHAN","ORG,PERSON"
5,GOOD AFTERNOON THANK YOU FOR CALLING WINDSTREA...,"WINDSTREAM,TANUSHA","ORG,PERSON"
6,GOOD EVENING THANK YOU FOR CALLING WINDSTREAM ...,"WINDSTREAM,ANSHU","ORG,PERSON"
7,THANK YOU FOR CALLING WINDSTREAM THIS IS LALIT,"LALIT,WINDSTREAM","PERSON,ORG"
8,GOOD MORNING MY NAME IS GEORGE,GEORGE,PERSON
9,THANK YOU FOR CALLING WINDSTREAM,WINDSTREAM,ORG


In [139]:
ls = []
for index,row in data.iterrows():
    words = row["WORD"]
    sent = row["SENTENCE"]
    ents = row["ENTITY"]
    word_list = words.split(",")
    ent_list = ents.split(",")
    tuple_list = []
    for word,ent in zip(word_list,ent_list):
        s_index = sent.index(word)
        e_index = s_index + len(word)
        temp = tuple([s_index,e_index,ent])
        tuple_list.append(temp)
    
    output = tuple([sent,{"entities":tuple_list}])
    ls.append(output)
    print(output)

('THANK YOU FOR CALLING WINDSTREAM  PRAVEEN HOW CAN I HELP HELLO THIS IS TYLER', {'entities': [(22, 32, 'ORG'), (34, 41, 'PERSON'), (71, 76, 'PERSON')]})
('THANK YOU FOR CALLING WINDSTREAM MY NAME IS RAVI HOW MAY I HELP YOU TODAY', {'entities': [(22, 32, 'ORG'), (44, 48, 'PERSON')]})
('THANK YOU FOR CALLING WINDSTREAM MY NAME IS VISHAL HOW MAY I HELP YOU TODAY', {'entities': [(22, 32, 'ORG'), (44, 50, 'PERSON')]})
('THANK YOU FOR CALLING WINDSTREAM MY NAME IS HANDA HOW MAY I HELP YOU TODAY', {'entities': [(22, 32, 'ORG'), (44, 49, 'PERSON')]})
('THANK YOU FOR CALLING WINDSTREAM MY NAME IS ROHAN HOW MAY I ASSIST YOU', {'entities': [(22, 32, 'ORG'), (44, 49, 'PERSON')]})
('GOOD AFTERNOON THANK YOU FOR CALLING WINDSTREAM MY NAME IS TANUSHA HOW MAY I HELP', {'entities': [(37, 47, 'ORG'), (59, 66, 'PERSON')]})
('GOOD EVENING THANK YOU FOR CALLING WINDSTREAM MY NAME IS ANSHU HOW CAN I HELP', {'entities': [(35, 45, 'ORG'), (57, 62, 'PERSON')]})
('THANK YOU FOR CALLING WINDSTREAM THIS IS LALIT

In [140]:
TRAIN_DATA = ls

In [141]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.load('en_core_web_md')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe('ner')
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)

# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

#Test your text
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Statring iteration 0
{'ner': 34.73562803927183}
Statring iteration 1
{'ner': 3.504599666316536}
Statring iteration 2
{'ner': 5.033344120655829}
Statring iteration 3
{'ner': 3.5994118829753603}
Statring iteration 4
{'ner': 1.2698106437627523}
Statring iteration 5
{'ner': 1.8388640000370766}
Statring iteration 6
{'ner': 0.0013822197728741142}
Statring iteration 7
{'ner': 8.24590871655061e-05}
Statring iteration 8
{'ner': 0.01370340201536004}
Statring iteration 9
{'ner': 1.0690628332587258e-06}
Statring iteration 10
{'ner': 0.0001133250015415882}
Statring iteration 11
{'ner': 3.235230602843122e-05}
Statring iteration 12
{'ner': 3.215816195793948e-05}
Statring iteration 13
{'ner': 0.01288970611386694}
Statring iteration 14
{'ner': 0.010472501972226643}
Statring iteration 15
{'ner': 6.142346503392796e-08}
Statring iteration 16
{'ner': 7.716149937302842e-08}
Statring iteration 17
{'ner': 3.5007897913807836e-07}
Statring iteration 18
{'ner': 2.1286448681362825e-07}
Statring iteration 19
{'ner

In [127]:
new_nlp = spacy.load("test")

In [130]:
text = "STEVE CONTACTED EARTHLINK TO OPEN UP A TICKET"
doc = new_nlp(text)

In [131]:
for x in doc.ents:
    print(x.text,x.label_)

STEVE PERSON
EARTHLINK ORG
